In [2]:
import os
import sys


module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
import numpy as np
import torch
from skimage import measure
from tqdm import tqdm, trange

from core.dataset import ScanNet
from core.integrate import ScalableTSDFVolume, FeatureFusionScalableTSDFVolume


In [5]:
tsdf_device = 'cuda:0'
tsdf_volume = FeatureFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=tsdf_device,
)


In [17]:
tsdf_load_dir = '/scratch/quanta/Experiments/feature-instance-fusion/scannet_scene0000_00/tsdf/'
scan_dir = '/scratch/quanta/Datasets/ScanNet/scans/scene0000_00/'

tsdf_pth = tsdf_load_dir + 'tsdf_volume.pt'
tsdf_unpruned_pth = tsdf_load_dir + 'tsdf_volume_unpruned.pt'

color_path = '/scratch/quanta/Experiments/feature-instance-fusion/scannet_scene0000_00/tsdf_feature_color/feats.pt'

gt_verts_high_res = scan_dir + 'scene0000_00_vh_clean.ply'
gt_verts_low_res = scan_dir + 'scene0000_00_vh_clean_2.ply'


In [20]:
from plyfile import PlyData
import numpy as np

low_res_ply = PlyData.read(gt_verts_low_res)
low_res_verts = np.vstack(
    [
        low_res_ply["vertex"].data["x"],
        low_res_ply["vertex"].data["y"],
        low_res_ply["vertex"].data["z"],
    ]
).transpose()
low_res_faces = np.vstack(low_res_ply["face"].data["vertex_indices"])

high_res_ply = PlyData.read(gt_verts_high_res)
high_res_verts = np.vstack(
    [
        high_res_ply["vertex"].data["x"],
        high_res_ply["vertex"].data["y"],
        high_res_ply["vertex"].data["z"],
    ]
).transpose()
high_res_faces = np.vstack(high_res_ply["face"].data["vertex_indices"])


In [7]:
tsdf_volume.load(tsdf_pth)
tsdf_volume.load_feats(color_path)


In [8]:
verts, faces = tsdf_volume.extract_mesh()


In [14]:
colors = tsdf_volume.extract_feat_on_grid(verts)[0]


In [15]:
import open3d as o3d

mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts),
    triangles=o3d.utility.Vector3iVector(faces),
)
mesh.vertex_colors = o3d.utility.Vector3dVector(colors)


In [16]:
o3d.io.write_triangle_mesh('output.ply', mesh)


True

## Calculate the percentage of vertex in tsdf volume
### Unpruned

In [23]:
low_res_verts.shape, high_res_verts.shape


((81369, 3), (1990518, 3))

In [26]:
from core.integrate.utils.voxel_ops import discretize_3d, discrete2hash
import torch

tsdf_volume_unpruned = ScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=tsdf_device,
)
tsdf_volume_unpruned.load(tsdf_unpruned_pth)

low_res_hash = discrete2hash(discretize_3d(torch.from_numpy(low_res_verts).to(tsdf_device), voxel_size=tsdf_volume.voxel_size, voxel_origin=tsdf_volume._vol_origin))

high_res_hash = discrete2hash(
    discretize_3d(torch.from_numpy(high_res_verts).to(tsdf_device),
                  voxel_size=tsdf_volume.voxel_size,
                  voxel_origin=tsdf_volume._vol_origin))


In [31]:
torch.isin(low_res_hash, tsdf_volume_unpruned._voxel_hash).float().mean().item(), torch.isin(high_res_hash, tsdf_volume_unpruned._voxel_hash).float().mean().item()


(0.9349998235702515, 0.9483813047409058)

### Pruned

In [32]:
tsdf_volume_pruned = ScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=tsdf_device,
)
tsdf_volume_pruned.load(tsdf_pth)


In [33]:
torch.isin(low_res_hash, tsdf_volume_pruned._voxel_hash).float().mean().item(), torch.isin(high_res_hash, tsdf_volume_pruned._voxel_hash).float().mean().item()


(0.7820914387702942, 0.8118645548820496)